In [ ]:
''' get the "ground truth" set of ENSPs that should be produced by funco/
    cereb for a given variant, randomly extracted from a raw vcf file '''

In [89]:
import vcfpy
import pandas as pd
pd.set_option('display.max_rows', 999)

In [90]:
def trim_funco_vcf(read_path, write_path):
    ''' gets rid of all the nonesense cerebra doesnt pick up '''
    reader = vcfpy.Reader.from_path(read_path)
    writer = vcfpy.Writer.from_path(write_path, reader.header)
    unwanted_class = ['COULD_NOT_DETERMINE', 'INTRON', 'FIVE_PRIME_UTR', 
                      'THREE_PRIME_UTR', 'IGR', 'FIVE_PRIME_FLANK', 'THREE_PRIME_FLANK', 'LINCRNA']

    for record in reader:
        funco = record.INFO.get('FUNCOTATION')[0]
    
        keep = True
        for elm in unwanted_class:
            if elm in funco:
                keep = False
    
        if keep:
            writer.write_record(record)

In [91]:
def get_indicies(pos, sorted_gtf):
    ''' get overlapping indicies from the gtf file '''
    keep_index_list = []
    for idx, row in sorted_gtf.iterrows():
        if row.start <= pos and row.end >= pos:
            keep_index_list.append(idx)
            
    return(keep_index_list)

In [92]:
def get_ensp_ids(indicies, sorted_gtf):
    ''' get the ENSP ids from the overlapping indicies in the gtf file '''
    sorted_gtf_trim = sorted_gtf.iloc[indicies]
    sorted_gtf_trim = sorted_gtf_trim.reset_index(drop=True)
    
    pids = []
    for idx, row in sorted_gtf_trim.iterrows():
        attr = row.attribute
        if 'protein_id' in attr:
            pid = attr.split('protein_id')[1].split(';')[0] # this is janky 
            pid_strip = pid.split('"')[1]
            pids.append(pid_strip)
    
    pids_split = [x.split('.')[0] for x in pids]
    ret = set(pids_split)
    
    return(ret)

In [93]:
''' MAIN STARTS HERE '''
# create trimmed funco vcf file
trim_funco_vcf('../tmp/G10_1001000340_benchmark.vcf', '../tmp/G10_1001000340_funco_trimmed.vcf')

# pull out a random line
! gshuf -n 1 ../tmp/G10_1001000340_funco_trimmed.vcf

##contig=<ID=chr1_KI270707v1_random,length=32032>


In [94]:
# make sure its in the raw vcf
! grep '43162920' ../tmp/G10_1001000340.vcf

chr22	43162920	.	A	G	1936.03	.	AC=2;AF=1.00;AN=2;DP=49;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=NaN;QD=30.24;SOR=1.473	GT:AD:DP:GQ:PL	1/1:0,49:49:99:1950,147,0


In [95]:
# subset the gtf file by gene name
! grep 'TSPO' /Users/lincoln.harris/code/cerebra/cerebra/tmp/ref/gencode.v27.chr_patch_hapl_scaff.basic.annotation.gtf > ../tmp/gene_sub.gtf

In [96]:
# get PIDs for the randomized position 
chrom = 22
pos = 43162920

sub_gtf = pd.read_csv('../tmp/gene_sub.gtf', sep='\t', 
            names=['chr', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])
sub_gtf_sorted = sub_gtf.sort_values('start')
sub_gtf_sorted = sub_gtf_sorted.reset_index(drop=True)

index_l = get_indicies(pos, sub_gtf_sorted)
ensp_l = get_ensp_ids(index_l, sub_gtf_sorted)
ensp_l = set(ensp_l)

print(ensp_l)

{'ENSP00000379563.4', 'ENSP00000345824.4', 'ENSP00000328973.4', 'ENSP00000268893.6', 'ENSP00000362252.2', 'ENSP00000362255.1', 'ENSP00000419985.1', 'ENSP00000463495.1', 'ENSP00000338004.3'}


In [63]:
#///////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////

In [73]:
# now lets find out what cereb found
    # cerebra missing like a lot of these...(looking at PIDs in ensp_l)
! grep 'ENSP00000338004' ../tmp/cereb_G10_1001000340_sub.csv

TSPO,"['ENSP00000379563.4:p.(Ala134Thr)', 'ENSP00000338004.3:p.(Met60Leu)', 'ENSP00000328973.4:p.(Ala134Thr)', 'ENSP00000328973.4:p.(Arg135=)', 'ENSP00000328973.4:p.(Glu76Ala)', 'ENSP00000405685.1:p.(Met60Leu)', 'ENSP00000405685.1:p.(Glu76Ala)', 'ENSP00000338004.3:p.(Ala134Thr)', 'ENSP00000379563.4:p.(Arg135=)', 'ENSP00000338004.3:p.(Thr147Ala)', 'ENSP00000379563.4:p.(Glu76Ala)', 'ENSP00000463495.1:p.(Arg31=)', 'ENSP00000463495.1:p.(Ala30Thr)', 'ENSP00000379563.4:p.(Thr147Ala)', 'ENSP00000338004.3:p.(Glu76Ala)', 'ENSP00000328973.4:p.(Thr147Ala)', 'ENSP00000328973.4:p.(Met60Leu)', 'ENSP00000463495.1:p.(Thr43Ala)', 'ENSP00000338004.3:p.(Arg135=)', 'ENSP00000379563.4:p.(Met60Leu)']"


In [62]:
# ok what does funco actually have? 
    # these are ENSP -> ENST converted IDs from BioMart
    
    # ok so funco actually missing quite a few of these as well 
enst_convert_list = ['ENST00000268893', 'ENST00000329563', 'ENST00000337554', 'ENST00000343736', 
                     'ENST00000373158', 'ENST00000373161', 'ENST00000396265', 'ENST00000470917', 
                     'ENST00000583777']

! grep 'ENST00000583777' ../tmp/G10_1001000340_benchmark.vcf 

chr22	43159416	.	A	C	183.60	.	AC=1;AF=0.500;AN=2;DP=45;ExcessHet=3.0103;FS=4.570;FUNCOTATION=[TSPO|hg38|chr22|43159416|43159416|MISSENSE||SNP|A|A|C|g.chr22:43159416A>C|ENST00000329563.8|+|2|404|c.178A>C|c.(178-180)Att>Ctt|p.I60L|0.683291770573566|CTACTCAGCCATGGGGTAGGT|TSPO_ENST00000337554.7_MISSENSE_p.I60L/TSPO_ENST00000583777.5_INTRON/TSPO_ENST00000396265.4_MISSENSE_p.T60P|||||||||||||||||||||||||||||false||];MLEAC=1;MLEAF=0.500;MQ=NaN;QD=4.08;SOR=0.140	GT:AD:DP:GQ:PL	0/1:37,8:45:99:191,0,1322
chr22	43161096	.	A	C	429.60	.	AC=1;AF=0.500;AN=2;DP=44;ExcessHet=3.0103;FS=5.497;FUNCOTATION=[TSPO|hg38|chr22|43161096|43161096|MISSENSE||SNP|A|A|C|g.chr22:43161096A>C|ENST00000329563.8|+|3|453|c.227A>C|c.(226-228)cAt>cCt|p.H76P|0.5785536159600998|GGCTTCACAGAGAAGGCTGTG|TSPO_ENST00000337554.7_MISSENSE_p.D76A/TSPO_ENST00000583777.5_FIVE_PRIME_UTR/TSPO_ENST00000396265.4_MISSENSE_p.D76A|||||||||||||||||||||||||||||false||];MLEAC=1;MLEAF=0.500;MQ=NaN;QD=9.76;SOR=1.817	GT:AD:DP:GQ:PL	0/1:30,14:44:99:4